In [39]:
import  numpy as np
import  matplotlib.pyplot as plt
import cv2
import pandas as pd
import requests
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.preprocessing import normalize
import pickle

In [40]:
with open('data3.pkl', 'rb') as f:
    data = pickle.load(f)

# open csv
tfidf = pd.read_csv('tfidf.csv')


In [41]:
image=None
import cv2

def adjust_brightness_and_contrast(image, brightness=0, contrast=0):
        # Alpha controls contrast; Beta controls brightness.
        alpha = 1 + contrast / 127
        beta = brightness
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        return adjusted_image

def preprocess(image_url):
    response = requests.get(image_url)
    image = cv2.imdecode(np.frombuffer(response.content, np.uint8), -1)
    if image is None:
        return None
    # image_height, image_width = image.shape[:2]




    #pixel normalisation
    def normalize_image(image):
        normalized_image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        return normalized_image

    image = normalize_image(image)
    

    #brightness and contrast adjustment
    def calculate_brightness_and_contrast(image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        mean, std_dev = cv2.meanStdDev(gray_image)
        return mean[0][0], std_dev[0][0]
 

    

    mean_brightness, std_dev_contrast = calculate_brightness_and_contrast(image)

    # Set a threshold for deciding whether to adjust brightness and contrast
    brightness_threshold = 20  
    contrast_threshold = 10 

    # Adjust brightness and contrast only if needed
    if mean_brightness < brightness_threshold or std_dev_contrast < contrast_threshold:
        brightness = 20  
        contrast = 20  
        adjusted_image = adjust_brightness_and_contrast(image, brightness=brightness, contrast=contrast)
    else:
        adjusted_image = image
    

    

    return image




IMAGE PREPROCESSING


In [42]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import vgg16
from sklearn.preprocessing import normalize

# Load pre-trained VGG16 model
vgg16_model = vgg16(pretrained=True)
vgg16_model.eval()

# Define a function to extract features using VGG16
def extract_features_vgg16(image):
    img = cv2.resize(image, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_tensor = transforms.ToTensor()(img)
    img_tensor = torch.unsqueeze(img_tensor, 0)
    with torch.no_grad():
        features = vgg16_model.features(img_tensor)
    features = torch.nn.functional.normalize(features, p=2, dim=1)
    features = features.squeeze().numpy()
    return features.flatten()

# Define a function to extract normalized features using VGG16
def extract_features_normalized_vgg16(image):
    extracted_features = extract_features_vgg16(image)
    normalized_features = normalize([extracted_features], norm='l2')
    return normalized_features[0]



C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [43]:

def cosine_similarity(vector_a, vector_b):
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [44]:
#  check similarity between two images
def check_similarity(image_url1, image_url2):
    image1 = preprocess(image_url1)
    image2 = preprocess(image_url2)
    if image1 is None or image2 is None:
        return None
    features1 = extract_features_normalized_vgg16(image1)
    features2 = extract_features_normalized_vgg16(image2)
    similarity = cosine_similarity(features1, features2)
    return similarity




In [45]:
#  preprorcess revierw texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer as ps

# Download the stopwords corpus if you haven't already
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

def preprocess_review_text(review_text):

    review_text = str(review_text)
    review_text = review_text.lower()
    review_text = re.sub(r'[^\w\s]', '', review_text)
    # remove stop words
    review_text = ' '.join([word for word in review_text.split() if word not in stop_words])
    # do stemming
    review_text = ' '.join([ps().stem(word) for word in review_text.split()])
    return review_text



# data['Review Text'] = data['Review Text'].apply(preprocess_review_text)


import numpy as np
from collections import Counter
import math

def tokenize(text):
    # Split text into tokens (words)
    return text.split()

def calculate_tf(text):
    # Calculate term frequency (TF) for each word in the text
    tokens = tokenize(text)
    word_count = Counter(tokens)
    total_words = len(tokens)
    tf = {word: count / total_words for word, count in word_count.items()}
    return tf

def calculate_idf(documents):
    # Calculate inverse document frequency (IDF) for each word
    total_documents = len(documents)
    all_words = set([word for document in documents for word in tokenize(document)])
    idf = {}
    for word in all_words:
        doc_count = sum([1 for document in documents if word in tokenize(document)])
        idf[word] = math.log10(total_documents / doc_count)
    return idf

def calculate_tfidf(text, idf):
    # Calculate TF-IDF for each word in the text using precomputed IDF values
    
    tf = calculate_tf(text)
    tfidf = {word: tf[word] * idf[word] for word in tf}
    return tfidf






data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.563436856848..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[0.008398145451137908, 0.0022459865717929435,..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[0.015007795081277633, 0.009100475550438039, ..."


In [46]:
data.head()

,Unnamed: 0,Image,Review Text,Image_Vectors
0,3452,[https://images-na.ssl-images-amazon.com/image...,love vintag spring vintag strat good tension g...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1205,[https://images-na.ssl-images-amazon.com/image...,work great guitar bench mat rug enough abus ta...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.563436856848..."
2,1708,[https://images-na.ssl-images-amazon.com/image...,use everyth acoust bass ukulel know smaller mo...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,2078,[https://images-na.ssl-images-amazon.com/image...,great price good qualiti didnt quit match radi...,"[[0.008398145451137908, 0.0022459865717929435,..."
4,801,[https://images-na.ssl-images-amazon.com/image...,bought bass split time primari bass dean edg m...,"[[0.015007795081277633, 0.009100475550438039, ..."


In [47]:
# name the first unnamed coluumn as id
data = data.rename(columns={'Unnamed: 0': 'id'})

In [48]:
#  make a tdidf matrix 
# 1. Calculate IDF for the dataset
idf = calculate_idf(data['Review Text'].tolist())

# 2. Calculate TF-IDF for each document
tfidf_matrix = []
for document in data['Review Text'].tolist():
    tfidf = calculate_tfidf(document, idf)
    tfidf_matrix.append(tfidf)

# 3. Convert the TF-IDF matrix to a DataFrame
tfidf = pd.DataFrame(tfidf_matrix)
tfidf = tfidf.fillna(0)
tfidf.head()



,love,vintag,spring,strat,good,tension,great,stabil,float,bridg,...,mayer,importantli,toneprint,stringthru,stopflair,biggi,accord,screenshot,amazoncom,piti
0,0.060264,0.232282,0.295898,0.090928,0.045850,0.13962,0.032906,0.13962,0.168018,0.097727,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.029902,0.00000,0.021461,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.057313,0.00000,0.041133,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.010730,0.00000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:

# Convert the query text to a dictionary of word frequenciesdef convertText
def convert_text_to_tfidf(text, idf):
    tfidf = calculate_tfidf(text, idf)
    return tfidf

In [74]:
def mostSimilarReviews(queryText):
    queryText=preprocess_review_text(queryText)
    
    idf=calculate_idf(data['Review Text'].tolist())
    queryText=" ".join([word for word in queryText.split() if word in idf])
    # Convert the query text to a dictionary of word frequencies
    queryText = convert_text_to_tfidf(queryText, idf)
    # Calculate the query vector
    queryVector = np.zeros(len(tfidf.columns))
    for word in queryText:
        if word in tfidf.columns:
            queryVector[tfidf.columns.get_loc(word)] = queryText[word]
    # Calculate the cosine similarity between the query vector and all the document vectors
    cosineSimilarities = {}
    for i in range(len(tfidf)):
        docVector = tfidf.iloc[i].values
        cosineSimilarities[i] = np.dot(queryVector, docVector) / (np.linalg.norm(queryVector) * np.linalg.norm(docVector))
    # Sort the cosine similarities
    sortedCosineSimilarities = sorted(cosineSimilarities.items(), key=lambda x: x[1], reverse=True)
    #  also return their respective scoerees
    indices= sortedCosineSimilarities[:3]
    scores=[i[1] for i in sortedCosineSimilarities[:3]]
    return indices, scores


In [81]:
#  cosine similarity between two texts
def textSimilarity(query_text1, query_text2):
    query_text1=preprocess_review_text(query_text1)
    query_text2=preprocess_review_text(query_text2)
    idf=calculate_idf(data['Review Text'].tolist())
    query_text1=" ".join([word for word in query_text1.split() if word in idf])
    query_text2=" ".join([word for word in query_text2.split() if word in idf])
    # Convert the query text to a dictionary of word frequencies
    queryText1 = convert_text_to_tfidf(query_text1, idf)
    queryText2 = convert_text_to_tfidf(query_text2, idf)
    # Calculate the query vector
    queryVector1 = np.zeros(len(tfidf.columns))
    queryVector2 = np.zeros(len(tfidf.columns))
    for word in queryText1:
        if word in tfidf.columns:
            queryVector1[tfidf.columns.get_loc(word)] = queryText1[word]
    for word in queryText2:
        if word in tfidf.columns:
            queryVector2[tfidf.columns.get_loc(word)] = queryText2[word]
    # Calculate the cosine similarity between the query vector and all the document vectors
    docVector1 = queryVector1
    docVector2 = queryVector2
    cosineSimilarity = np.dot(docVector1, docVector2) / (np.linalg.norm(docVector1) * np.linalg.norm(docVector2))
    return cosineSimilarity

In [86]:


# print an example
query_text = "I love the product"
mostSimilarReviews(query_text)[0][0]


(118, 0.46496524195976735)

In [69]:
#  print the review text of the top 5 similar reviews to the qeury text
for i, in mostSimilarReviews('I have been using Fender locking tuners for about five years on variousstrats and teles. Definitely helps with tuning stability and way faster to restring ifthere is a break.'):
    print(data.iloc[i[0]]['Review Text'])

use fender lock tuner five year variou strat tele definit help tune stabil way faster restr break
went fender chrome nonlock fender gold lock made guitar look beauti play beauti think lock tuner way go new lock tuner look youtub instruct
tele perfect thank much


In [67]:
#  find the most similar images top 5 given an image url
def find_similar_images(image_url):
    image = preprocess(image_url)
    if image is None:
        return None
    features = extract_features_normalized_vgg16(image)
    # print(features)

    similarities = {}
    for i ,x in enumerate(data['Image_Vectors']):
        vectors=x
        max_sim=0
        for vector in vectors:
            # print(vector)   
            
            similarity = cosine_similarity(features, vector)
            if similarity > max_sim:
                max_sim = similarity

        similarities[i] = max_sim
    similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return similarities[:3]


find_similar_images("https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg")   

[(575, 0.5101304632627585),
 (652, 0.43318719702258773),
 (516, 0.40847325497715176)]

In [99]:
# take the input image link from user
image_urlx = input("Enter the image url: ")
# #  take users input for the review
review = input("Enter the review: ")

# composite dictionary to store these results
composite = {}


#  find the most similar images top 3 images 
similar_images = find_similar_images(image_urlx)
print("Using Image retrieval")
for i in similar_images:
    print("Image URL",image_url:=data.iloc[i[0]]['Image'])
    print("Review Text: ", data.iloc[i[0]]['Review Text'])
    print("Image Similarity Score: ",i[1] )
    review1= data.iloc[i[0]]['Review Text']
    print("Review Similarity Score: ",textSimilarity(review,review1)) 
    print("Composite Similarity Score: ",(i[1]+textSimilarity(review,review1))/2)
    composite[i[0]] =(i[1]+textSimilarity(review,review1))/2

print("Using Review retrieval")
similarReviews1 = mostSimilarReviews(review)[0]
for i in similarReviews1:
    print("Image URL",data.iloc[i[0]]['Image'])
    print("Review Text: ", data.iloc[i[0]]['Review Text'])
    print("Review Similarity Score: ",i[1] )
    image_urls1= data.iloc[i[0]]['Image']
    maxImageSim=0
    for x in image_urls1:
        maxImageSim=max(maxImageSim,check_similarity(image_urlx,x))
    print("Image Similarity Score: ",maxImageSim)
    print("Composite Similarity Score: ",(i[1]+maxImageSim)/2)
    composite[i[0]] =(i[1]+textSimilarity(review,review1))/2

#  sort the composite dictionary




Using Image retrieval
Image URL ['https://images-na.ssl-images-amazon.com/images/I/61n284XL9HL._SY88.jpg']
Review Text:  easi heck put opinion better sperzel took liter 10 minut put mim strat thing ill say probabl need setup remov tuner remov string guid tree headstock go chang angl bridg littl bit awar
Image Similarity Score:  0.5101304632627585
Review Similarity Score:  0.05949794376613932
Composite Similarity Score:  0.2848142035144489
Image URL ['https://images-na.ssl-images-amazon.com/images/I/719-SDMiOoL._SY88.jpg']
Review Text:  lock tuner look great keep tune good qualiti materi construct excel upgrad guitar drill addit hole instal neck alreadi come predril hole drop right otherwis need buy guitar tuner pin drill jig also avail amazon
Image Similarity Score:  0.43318719702258773
Review Similarity Score:  0.12725617802413502
Composite Similarity Score:  0.2802216875233614
Image URL ['https://images-na.ssl-images-amazon.com/images/I/51V0K2tgLUL._SY88.jpg', 'https://images-na.ssl-

IndexError: index 0 is out of bounds for axis 0 with size 0

In [105]:
composite = dict(sorted(composite.items(), key=lambda item: item[1], reverse=True))
print(composite)
print("Composite Similarity")
for key, value in composite.items():
    print("Image URL",data.iloc[key]['Image'])
    print("Composite Similarity Score: ",value)
    print("Review Text: ", data.iloc[key]['Review Text'])


{754: 0.5233423746109156, 575: 0.2848142035144489, 652: 0.2802216875233614, 516: 0.22757900209949142, 619: 0.15250819359772808, 947: 0.10916684924948701}
Composite Similarity
Image URL ['https://images-na.ssl-images-amazon.com/images/I/71bztfqdg+L._SY88.jpg']
Composite Similarity Score:  0.5233423746109156
Review Text:  use fender lock tuner five year variou strat tele definit help tune stabil way faster restr break
Image URL ['https://images-na.ssl-images-amazon.com/images/I/61n284XL9HL._SY88.jpg']
Composite Similarity Score:  0.2848142035144489
Review Text:  easi heck put opinion better sperzel took liter 10 minut put mim strat thing ill say probabl need setup remov tuner remov string guid tree headstock go chang angl bridg littl bit awar
Image URL ['https://images-na.ssl-images-amazon.com/images/I/719-SDMiOoL._SY88.jpg']
Composite Similarity Score:  0.2802216875233614
Review Text:  lock tuner look great keep tune good qualiti materi construct excel upgrad guitar drill addit hole ins